In [ ]:
%config SqlMagic.autopolars = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
%load_ext sql

# Dépendances


In [ ]:
import json
import math
import os
from datetime import datetime, timedelta
from itertools import product
from pathlib import Path
from zoneinfo import ZoneInfo

import branca.colormap as bcm
import duckdb
import folium
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import polars as pl
import polars_h3 as plh3
import polars_st as st
import shapely
from dotenv import load_dotenv
from folium import plugins
from sqlalchemy import create_engine

# Configuration


In [ ]:
USE_CACHED_ARTIFACTS = True

In [ ]:
load_dotenv()
DB_URL = os.environ.get("DB_URL")

In [ ]:
AOM_SIRET = "28750007800020"

In [ ]:
OUTPUT_PATH = Path("outputs_idfm")

In [ ]:
labels_map = {
    "month": "Mois",
    "num_journeys": "Nombre de journeys",
    "share_journeys": "% des journeys",
    "num_journeys_incentived": "Nombre de journey avec incitation",
    "num_journeys_with_incentive": "Nombre de journey avec incitation",
    "num_journeys_intra_territory_incentived_trips": "Nombre de journeys incitées intra",
    "num_journeys_inter_territory_incentived_trips": "Nombre de journeys incitées inter",
    "operator": "Opérateur",
    "incentive_amount_avg": "Incitation moyenne",
    "driver_revenue_avg": "Revenu moyen conducteur",
    "passenger_contribution_avg": "Contribution moyenne passager",
    "incentive_amount_intra_avg": "Incitation moyenne intra",
    "driver_revenue_intra_avg": "Revenu moyen conducteur intra",
    "passenger_contribution_intra_avg": "Contribution moyenne passager intra",
    "incentive_amount_inter_avg": "Incitation moyenne inter",
    "driver_revenue_inter_avg": "Revenu moyen conducteur inter",
    "passenger_contribution_inter_avg": "Contribution moyenne passager inter",
    "incentive_amount_per_km_avg": "Montant moyen d'incitation par km",
    "passenger_contribution_per_km_avg": "Contribution moyenne passager par km",
    "driver_revenue_per_km_avg": "Revenu moyen conducteur par km",
    "week": "Semaine",
    "month": "Mois",
    "distance_avg": "Distance moyenne",
    "distance_incentived_trips_avg": "Distance moyenne [km] - journeys avec incentives",
    "campaign_type": "Campagne",
    "distance": "Distance",
    "num_journeys_with_aom_incentive": "Nombre de journeys incitées par l'AOM",
    "num_journeys_with_operator_incentive": "Nombre de journeys incitées par un opérateur",
    "num_journeys_intra_territory": "Nombre de journeys intra-territoire",
    "num_journeys_inter_territory": "Nombre de journeys inter-territoires",
    "share_journeys_intra_territory": "% de journeys intra-territoire",
    "share_journeys_inter_territory": "% de journeys inter-territoires",
    "share_drivers": "% des conducteurs",
    "num_trips": "Nombre de trips",
    "is_intra_driver": "Conducteur intra",
    "driver_campaign_type": "Type de campagne du conducteur",
    "passenger_campaign_type": "Type de campagne du passager",
    "drivers_share": "% des conducteurs",
    "week_number": "Semaine n°",
    "passengers_share": "% des passagers",
}

## duckdb


In [ ]:
conn = duckdb.connect(
    "db.duckdb",
    config={"memory_limit": "16GiB", "threads": 4, "preserve_insertion_order": False},
)
%sql conn --alias duckdb

In [ ]:
%%sql
INSTALL spatial;

LOAD spatial;

# Queries


In [ ]:
SQL_ENGINE = create_engine(DB_URL)

## Journeys


In [ ]:
SQL = """
with idfm_perimeter as 
(
select
	p.arr,
	max(p.com) as com,
	max(p.geom_simple) as geom_simple
from
	geo.perimeters p
where
	p.reg = '11'
	and year = 2024
group by
	1
),
geo_filtered as (
select
	g.carpool_id,
	g.start_geo_code,
	g.end_geo_code,
	(substring(g.start_geo_code for 2) in ('94', '78', '92', '91', '75', '93', '95', '77')
		and substring(g.end_geo_code for 2) in ('94', '78', '92', '91', '75', '93', '95', '77')) as is_fully_inside_campaign_area
from
	carpool_v2.geo g
where
	(substring(g.start_geo_code for 2) in ('94', '78', '92', '91', '75', '93', '95', '77')
		or substring(g.end_geo_code for 2) in ('94', '78', '92', '91', '75', '93', '95', '77'))
			and g.updated_at >= '2024-09-01'
),
first_trip as (
select
    driver_identity_key,
    min(c.start_datetime) as first_trip_datetime
from carpool_v2.carpools c
group by 1
),
first_trip_passengers as (
select
    passenger_identity_key,
    min(c.start_datetime) as first_trip_datetime
from carpool_v2.carpools c
group by 1
),
incentives as (
select
	oi.carpool_id,
	sum(oi.amount) as incentive_amount,
    sum(oi.amount) filter (where siret='28750007800020') as amount_aom,
	array_agg(distinct oi.siret) as incentive_sirets
from
	carpool_v2.operator_incentives oi
inner join geo_filtered g on
	oi.carpool_id = g.carpool_id
where amount>0
group by
	1
),
journeys as 
(
select
	c."_id",
	c.operator_id,
	c.operator_journey_id,
	c.operator_trip_id,
    c.driver_identity_key,
    ft.first_trip_datetime,
    c.passenger_identity_key,
    ftp.first_trip_datetime as passenger_first_trip_datetime,
	c.start_datetime,
	c.end_datetime,
	c.distance,
	c.driver_revenue,
	c.passenger_contribution,
	i.incentive_amount,
    i.amount_aom,
	i.incentive_sirets,
	c.start_position,
	c.end_position,
    c.passenger_seats,
    c.passenger_travelpass_name,
    c.passenger_travelpass_user_id,
    is_fully_inside_campaign_area,
	ST_MAKELINE(c.start_position::geometry,c.end_position::geometry) as journey_line	
from
	carpool_v2.carpools c
inner join geo_filtered g on
	c."_id" = g.carpool_id
left join incentives i on
	c."_id" = i.carpool_id
left join first_trip ft on ft.driver_identity_key=c.driver_identity_key
left join first_trip_passengers ftp on ftp.passenger_identity_key=c.passenger_identity_key
left join carpool_v2.status s on s."carpool_id"=c."_id" 
where
	(c.start_datetime between '2024-09-01' and '2025-07-20')
    and s.acquisition_status='processed'
    and s.fraud_status='passed'
    and s.anomaly_status='passed'
    )
SELECT
    j.*,
    CASE WHEN p.l_arr = p.country THEN p.l_country ELSE p.l_arr END as start_com,
    CASE WHEN p2.l_arr = p2.country THEN p2.l_country ELSE p2.l_arr END as end_com
from journeys j
left join carpool_v2.geo g on j."_id"=g."carpool_id"
left join geo.perimeters p on g."start_geo_code"=p.arr and p.year=2024
left join geo.perimeters p2 on g."end_geo_code"=p2.arr and p2.year=2024
"""

In [ ]:
if USE_CACHED_ARTIFACTS:
    df_journeys_raw = pl.read_parquet("df_journeys_raw.parquet")
else:
    df_journeys_raw = pl.read_database(
        query=SQL,
        connection=SQL_ENGINE,
        schema_overrides={
            "passenger_travelpass_name": pl.String,
            "passenger_travelpass_user_id": pl.String,
        },
    )
    df_journeys_raw.write_parquet("df_journeys_raw.parquet", compression_level=6)

In [ ]:
df_journeys_raw.schema

In [ ]:
df_journeys_raw.estimated_size() / 1e7

In [ ]:
df_journeys_raw.head()

In [ ]:
df_journeys_raw.describe()

## Opérateurs


In [ ]:
df_operators = pl.read_database(
    query="""
SELECT
    "_id",
    "name",
    "siret"
from operator.operators
where deleted_at is null
and name!='BlaBlaCar'
""",
    connection=SQL_ENGINE,
)

In [ ]:
df_operators

# Reseau IDFM


In [ ]:
%%sql
CREATE TABLE
  IF NOT EXISTS gares_idfm AS
SELECT
  id_gares,
  nom_gares,
  nom_so_gar,
  nom_su_gar,
  id_ref_zdc,
  nom_zdc,
  id_ref_zda,
  nom_zda,
  idrefliga,
  idrefligc,
  res_com,
  indice_lig,
  mode,
  tertrain,
  terrer,
  termetro,
  tertram,
  terval,
  exploitant,
  idf,
  ST_FlipCoordinates (geom) AS geom --  EPSG:4326 coordinate system (WGS84), with [latitude, longitude] axis order
FROM
  ST_Read (
    '/home/toto/Documents/Grammaticorp/covoiturage/preuve-covoiturage/notebooks/analytics/campaigns/emplacement-des-gares-idf.geojson'
  )
WHERE
  mode IN ('TRAIN', 'RER');

In [ ]:
%%sql
SELECT
  COUNT(*)
FROM
  gares_idfm;

In [ ]:
%%sql
CREATE INDEX IF NOT EXISTS gares_geom_index ON gares_idfm USING RTREE (geom);

In [ ]:
%%sql
FROM
  (DESCRIBE gares_idfm);

In [ ]:
%%sql df_idfm_stations <<
SELECT
    *,
    ST_asText(geom) as geom_wkt
FROM gares_idfm

In [ ]:
df_idfm_stations

# Identification incitateurs


In [ ]:
df_journeys_raw = df_journeys_raw.with_columns(
    pl.col("incentive_sirets").list.contains(AOM_SIRET).alias("incentived_by_aom"),
    (
        pl.col("incentive_sirets")
        .list.set_intersection(df_operators["siret"].to_list())
        .list.len()
        > 0
    ).alias("incentived_by_operator"),
)

# Traitements geo


In [ ]:
df_journeys_raw = df_journeys_raw.with_columns(
    pl.col("start_position")
    .map_elements(lambda x: shapely.from_wkb(x).wkt, return_dtype=pl.String)
    .alias("start_pos"),
    pl.col("end_position")
    .map_elements(lambda x: shapely.from_wkb(x).wkt, return_dtype=pl.String)
    .alias("end_pos"),
)

# Filtrage des journeys sans incitations


In [ ]:
df_journeys = df_journeys_raw.filter((pl.col("incentive_amount") > 0))

# Création de la table des journeys sur duckdb


In [ ]:
%%sql
CREATE TABLE
  if NOT EXISTS journeys_raw AS
SELECT
  _id,
  operator_id,
  operator_journey_id,
  operator_trip_id,
  driver_identity_key,
  first_trip_datetime,
  passenger_identity_key,
  passenger_first_trip_datetime,
  start_datetime,
  end_datetime,
  distance,
  driver_revenue,
  passenger_contribution,
  incentive_amount,
  amount_aom,
  incentive_sirets,
  start_position,
  end_position,
  passenger_seats,
  is_fully_inside_campaign_area,
  journey_line,
  start_com,
  end_com,
  incentived_by_aom,
  incentived_by_operator,
  ST_FlipCoordinates (ST_GeomFromText (start_pos)) AS start_pos,
  ST_FlipCoordinates (ST_GeomFromText (end_pos)) AS end_pos
FROM
  df_journeys_raw

In [ ]:
%%sql
CREATE INDEX IF NOT EXISTS start_pos_idx ON journeys_raw USING RTREE (start_pos);

CREATE INDEX IF NOT EXISTS end_pos_idx ON journeys_raw USING RTREE (end_pos);

In [ ]:
%%sql
FROM
  (DESCRIBE journeys_raw)

In [ ]:
%%sql
SELECT
  *
FROM
  journeys_raw
LIMIT
  5

In [ ]:
%%sql
SELECT
  COUNT(*)
FROM
  journeys_raw

# Statistiques globales


In [ ]:
incentived_trip_filter_expr = pl.col("incentive_amount") > 0

agg_expressions = [
    pl.col("_id").n_unique().alias("num_journeys"),
    pl.col("_id")
    .filter(pl.col("is_fully_inside_campaign_area"))
    .n_unique()
    .alias("num_journeys_intra_territory"),
    pl.col("_id")
    .filter(pl.col("is_fully_inside_campaign_area") & incentived_trip_filter_expr)
    .n_unique()
    .alias("num_journeys_intra_territory_incentived_trips"),
    pl.col("_id")
    .filter(incentived_trip_filter_expr)
    .n_unique()
    .alias("num_journeys_incentived"),
    pl.col("_id")
    .filter(pl.col("incentived_by_aom"))
    .n_unique()
    .alias("num_journeys_with_aom_incentive"),
    pl.col("_id")
    .filter(pl.col("incentived_by_operator"))
    .n_unique()
    .alias("num_journeys_with_operator_incentive"),
    pl.col("_id")
    .filter(pl.col("is_fully_inside_campaign_area"))
    .n_unique()
    .alias("num_journeys_intra"),
    (pl.col("distance") / 1000).mean().alias("distance_avg"),
    (pl.col("distance").filter(incentived_trip_filter_expr) / 1000)
    .mean()
    .alias("distance_incentived_trips_avg"),
    (pl.col("incentive_amount").mean() / 100).alias("incentive_amount_avg"),
    (pl.col("passenger_contribution").filter(incentived_trip_filter_expr.not_()) / 100)
    .mean()
    .alias("passenger_contribution_avg"),
    (pl.col("passenger_contribution").filter(incentived_trip_filter_expr) / 100)
    .mean()
    .alias("passenger_contribution_incentived_trips_avg"),
    (
        pl.col("driver_revenue").filter(incentived_trip_filter_expr.not_()).mean() / 100
    ).alias("driver_revenue_avg"),
    (pl.col("driver_revenue").filter(incentived_trip_filter_expr).mean() / 100).alias(
        "driver_revenue_incentived_trips_avg"
    ),
    (
        pl.col("incentive_amount")
        .filter(pl.col("is_fully_inside_campaign_area"))
        .mean()
        / 100
    ).alias("incentive_amount_intra_avg"),
    (
        pl.col("passenger_contribution")
        .filter(pl.col("is_fully_inside_campaign_area") & incentived_trip_filter_expr)
        .mean()
        / 100
    ).alias("passenger_contribution_intra_avg"),
    (
        pl.col("driver_revenue")
        .filter(pl.col("is_fully_inside_campaign_area") & incentived_trip_filter_expr)
        .mean()
        / 100
    ).alias("driver_revenue_intra_avg"),
    (
        pl.col("incentive_amount")
        .filter(
            pl.col("is_fully_inside_campaign_area").not_() & incentived_trip_filter_expr
        )
        .mean()
        / 100
    ).alias("incentive_amount_inter_avg"),
    (
        pl.col("passenger_contribution")
        .filter(
            pl.col("is_fully_inside_campaign_area").not_() & incentived_trip_filter_expr
        )
        .mean()
        / 100
    ).alias("passenger_contribution_inter_avg"),
    (
        pl.col("driver_revenue")
        .filter(
            pl.col("is_fully_inside_campaign_area").not_() & incentived_trip_filter_expr
        )
        .mean()
        / 100
    ).alias("driver_revenue_inter_avg"),
    (10 * (pl.col("incentive_amount") / pl.col("distance")))
    .mean()
    .alias("incentive_amount_per_km_avg"),
    (
        10
        * (pl.col("passenger_contribution") / pl.col("distance")).filter(
            incentived_trip_filter_expr.not_()
        )
    )
    .mean()
    .alias("passenger_contribution_per_km_avg"),
    (
        10
        * (pl.col("passenger_contribution") / pl.col("distance")).filter(
            incentived_trip_filter_expr
        )
    )
    .mean()
    .alias("passenger_contribution_per_km_incentived_trips_avg"),
    (
        10
        * (pl.col("driver_revenue") / pl.col("distance")).filter(
            incentived_trip_filter_expr.not_()
        )
    )
    .mean()
    .alias("driver_revenue_per_km_avg"),
    (
        10
        * (pl.col("driver_revenue") / pl.col("distance")).filter(
            incentived_trip_filter_expr
        )
    )
    .mean()
    .alias("driver_revenue_per_km_incentived_trips_avg"),
    pl.col("driver_identity_key").n_unique().alias("number_of_unique_driver"),
    pl.col("passenger_identity_key").n_unique().alias("number_of_unique_passenger"),
]

In [ ]:
df_stats_by_month = (
    df_journeys_raw.group_by(pl.col("start_datetime").dt.truncate("1mo").alias("month"))
    .agg(agg_expressions)
    .sort(pl.col("month"))
)

In [ ]:
df_stats_by_week = (
    df_journeys_raw.filter(
        pl.col("start_datetime") <= datetime(2025, 7, 20, tzinfo=ZoneInfo("GMT"))
    )
    .group_by(pl.col("start_datetime").dt.truncate("1w").alias("week"))
    .agg(agg_expressions)
    .sort(pl.col("week"))
)

## Nombre de journeys


In [ ]:
with pl.Config(set_fmt_str_lengths=120, set_tbl_width_chars=1000):
    print(
        df_journeys_raw.select(
            pl.col("_id").n_unique().alias("Nombre de journeys"),
            pl.col("_id")
            .filter(pl.col("incentive_amount") > 0)
            .n_unique()
            .alias("Nombre de journeys avec incitation"),
            (
                100
                * pl.col("_id").filter(pl.col("incentive_amount") > 0).n_unique()
                / pl.col("_id").n_unique()
            ).alias("% journeys avec incitation"),
            pl.col("_id")
            .filter(pl.col("incentived_by_aom"))
            .n_unique()
            .alias("Nombre de journeys avec incitation AOM"),
            (
                100
                * pl.col("_id").filter(pl.col("incentived_by_aom")).n_unique()
                / pl.col("_id").n_unique()
            ).alias("% journeys avec incitation AOM"),
            pl.col("_id")
            .filter(pl.col("incentived_by_operator"))
            .n_unique()
            .alias("Nombre de journeys avec incitation opérateur"),
            (
                100
                * pl.col("_id").filter(pl.col("incentived_by_operator")).n_unique()
                / pl.col("_id").n_unique()
            ).alias("% journeys avec incitation opérateur"),
        )
        .with_columns(pl.selectors.all().round(2))
        .unpivot()
    )

### Evolution


#### Globale


In [ ]:
def create_num_journeys_fig(
    df: pl.DataFrame,
    x_col: str = "month",
    title: str = "IDFM - Nombre de journeys par mois",
) -> go.Figure:
    traces = []
    max_y = 0

    df_grouped = df.group_by()
    for name in [
        "num_journeys",
        "num_journeys_incentived",
        "num_journeys_with_aom_incentive",
        "num_journeys_with_operator_incentive",
    ]:
        trace = go.Scatter(
            x=df[x_col],
            y=df[name],
            mode="lines+text" if name == "num_journeys" else "lines",
            textposition="top center",
            text=df[name] if name == "num_journeys" else None,
            name=labels_map.get(name, name),
        )
        traces.append(trace)
        max_y = max(max_y, df[name].max())

    fig = go.Figure(traces)

    fig.update_layout(
        template="simple_white",
        title=title,
        legend_orientation="h",
        legend_y=0.6,
        legend_yref="container",
    )
    fig.update_xaxes(title="Mois" if x_col == "month" else "Semaine")
    fig.update_yaxes(range=[0, max_y * 1.2], showgrid=True, title="Nombre de journeys")

    return fig

In [ ]:
fig_journeys_by_month = create_num_journeys_fig(df_stats_by_month)
fig_journeys_by_month.show()

fig_journeys_by_month.write_html(OUTPUT_PATH / "fig_journeys_par_mois.html")
fig_journeys_by_month.write_image(
    OUTPUT_PATH / "fig_journeys_par_mois.svg", width=1280, height=720
)

#### Opérateur incitateurs


In [ ]:
fig_journeys_by_operator = px.line(
    df_journeys.explode("incentive_sirets")
    .join(df_operators, left_on="incentive_sirets", right_on="siret", how="left")
    .group_by(["name", pl.col("start_datetime").dt.truncate("1mo")])
    .agg(pl.col("operator_journey_id").n_unique().alias("num_journeys"))
    .rename({"name": "operator", "start_datetime": "month"})
    .sort("month"),
    x="month",
    y="num_journeys",
    color="operator",
    template="simple_white",
    labels=labels_map,
    title="Nombre de journeys incités par opérateur",
)
fig_journeys_by_operator.update_yaxes(showgrid=True)
fig_journeys_by_operator.show()


fig_journeys_by_operator.write_html("outputs_idfm/fig_journeys_par_operateur_mois.html")
fig_journeys_by_operator.write_image(
    "outputs_idfm/fig_journeys_par_operateur_mois.svg", width=1280, height=720
)

#### intra vs inter


In [ ]:
fig_journeys_by_journey_type = px.line(
    df_stats_by_week.with_columns(
        (pl.col("num_journeys") - pl.col("num_journeys_intra_territory")).alias(
            "num_journeys_inter_territory"
        )
    ),
    x="week",
    y=["num_journeys_intra_territory", "num_journeys_inter_territory"],
    template="simple_white",
    labels=labels_map,
    title="Nombre de journeys par type de trajets",
)
fig_journeys_by_journey_type.update_traces(
    {"name": labels_map["num_journeys_intra_territory"]},
    selector={"name": "num_journeys_intra_territory"},
)
fig_journeys_by_journey_type.update_traces(
    {"name": labels_map["num_journeys_inter_territory"]},
    selector={"name": "num_journeys_inter_territory"},
)
fig_journeys_by_journey_type.update_yaxes(showgrid=True, title="Nombre de journeys")
fig_journeys_by_journey_type.update_layout(
    legend_title="", legend_orientation="h", legend_y=0.7, legend_yref="container"
)
fig_journeys_by_journey_type.show()


fig_journeys_by_journey_type.write_html(
    OUTPUT_PATH / "fig_journeys_par_type_semaine.html"
)
fig_journeys_by_journey_type.write_image(
    OUTPUT_PATH / "fig_journeys_par_type_semaine.svg", width=1280, height=720
)

In [ ]:
fig_journeys_by_journey_type_incentived_trips = px.line(
    df_stats_by_week.with_columns(
        (
            pl.col("num_journeys_incentived")
            - pl.col("num_journeys_intra_territory_incentived_trips")
        ).alias("num_journeys_inter_territory_incentived_trips")
    ),
    x="week",
    y=[
        "num_journeys_intra_territory_incentived_trips",
        "num_journeys_inter_territory_incentived_trips",
    ],
    template="simple_white",
    labels=labels_map,
    title="Nombre de journeys par type de trajets - Trajets incités",
)

for e in [
    "num_journeys_intra_territory_incentived_trips",
    "num_journeys_inter_territory_incentived_trips",
]:
    fig_journeys_by_journey_type_incentived_trips.update_traces(
        {"name": labels_map.get(e)},
        selector={"name": e},
    )
fig_journeys_by_journey_type_incentived_trips.update_yaxes(
    showgrid=True, title="Nombre de journeys"
)
fig_journeys_by_journey_type_incentived_trips.update_layout(
    legend_title="", legend_orientation="h", legend_y=0.7, legend_yref="container"
)
fig_journeys_by_journey_type_incentived_trips.show()


fig_journeys_by_journey_type_incentived_trips.write_html(
    "outputs_idfm/fig_journeys_par_type_semaine_trajets_incites.html"
)
fig_journeys_by_journey_type_incentived_trips.write_image(
    "outputs_idfm/fig_journeys_par_type_semaine_trajets_incites.svg",
    width=1280,
    height=720,
)

## Par opérateurs


In [ ]:
px.line(
    df_journeys.group_by(["operator_id", pl.col("start_datetime").dt.truncate("1w")])
    .agg(pl.col("_id").n_unique().alias("num_journeys"))
    .join(df_operators, left_on="operator_id", right_on="_id", validate="m:1")
    .sort(["start_datetime", "operator_id"]),
    x="start_datetime",
    y="num_journeys",
    color="name",
    labels=labels_map,
    template="simple_white",
    title="Nombre de journeys par opérateur<br><sub>Uniquement les trajets incités</sub>",
)

## Distance


In [ ]:
fig_distance_by_month = px.line(
    df_stats_by_month.with_columns(
        pl.col("distance_avg").round(1).alias("distance_avg_fmt")
    ),
    x="month",
    y="distance_avg",
    text="distance_avg_fmt",
    template="simple_white",
    labels=labels_map,
    title="Evolution de la distance moyenne par trajets",
)
fig_distance_by_month.update_traces(textposition="top center")
fig_distance_by_month.update_yaxes(
    range=[0, df_stats_by_month["distance_avg"].max() * 1.1], zeroline=True
)
fig_distance_by_month.show()

## Prix, revenus et incitations


### Trajets Incités


In [ ]:
def create_scatter_fig_prices(
    df: pl.DataFrame,
    stats_cols: list[str],
    x_col: str,
    title: str,
    labels_map: dict[str, str],
    x_title: str = "Montant (euros)",
) -> go.Figure:
    traces = []
    for name in stats_cols:
        trace = go.Scatter(
            x=df[x_col],
            y=df[name],
            name=labels_map.get(name, name),
            mode="lines+markers",
            marker_size=4,
        )
        traces.append(trace)
    fig = go.Figure(traces)
    fig.update_layout(
        template="simple_white",
        title=title,
        legend_orientation="h",
        legend_y=0.7,
        legend_yref="container",
    )

    max_y = df.select(stats_cols).max().max_horizontal().item()

    fig.update_yaxes(
        range=[0, max_y * 1.2],
        title=x_title,
        showgrid=True,
        gridwidth=2,
        ticksuffix="€",
    )
    fig.update_xaxes(title="Mois" if x_col == "month" else "Semaine")

    return fig

In [ ]:
fig_prices_by_week = create_scatter_fig_prices(
    df_stats_by_week,
    [
        "incentive_amount_avg",
        "passenger_contribution_incentived_trips_avg",
        "driver_revenue_incentived_trips_avg",
    ],
    "week",
    (
        "Montants moyens par trajet des incitations,"
        "<br>contributions passagers et revenus conducteurs - Trajets incités"
    ),
    {
        **labels_map,
        "passenger_contribution_incentived_trips_avg": "Contribution moyenne passager",
        "driver_revenue_incentived_trips_avg": "Revenu moyen conducteur",
    },
)
fig_prices_by_week.show()


fig_prices_by_week.write_html(OUTPUT_PATH / "fig_prix_par_semaine.html")
fig_prices_by_week.write_image(
    OUTPUT_PATH / "fig_prix_par_semaine.svg", width=1280, height=720
)

#### Intra


In [ ]:
fig_prices_by_week_intra = create_scatter_fig_prices(
    df_stats_by_week,
    [
        "incentive_amount_intra_avg",
        "passenger_contribution_intra_avg",
        "driver_revenue_intra_avg",
    ],
    "week",
    (
        "Montants moyens par trajet <b>intra</b> des incitations,"
        "<br>contributions passagers et revenus conducteurs"
    ),
    labels_map,
)
fig_prices_by_week_intra.show()


fig_prices_by_week_intra.write_html(OUTPUT_PATH / "fig_prix_intra_par_semaine.html")
fig_prices_by_week_intra.write_image(
    OUTPUT_PATH / "fig_prix_intra_par_semaine.svg", width=1280, height=720
)

#### Inter


In [ ]:
fig_prices_by_week_inter = create_scatter_fig_prices(
    df_stats_by_week,
    [
        "incentive_amount_inter_avg",
        "passenger_contribution_inter_avg",
        "driver_revenue_inter_avg",
    ],
    "week",
    (
        "Montants moyens par trajet <b>inter</b> des incitations,"
        "<br>contributions passagers et revenus conducteurs"
    ),
    labels_map,
)
fig_prices_by_week_inter.show()


fig_prices_by_week_inter.write_html(OUTPUT_PATH / "fig_prix_inter_par_semaine.html")
fig_prices_by_week_inter.write_image(
    OUTPUT_PATH / "fig_prix_inter_par_semaine.svg", width=1280, height=720
)

#### Au kilomètre


In [ ]:
fig_prices_per_km_by_week = create_scatter_fig_prices(
    df_stats_by_week,
    [
        "incentive_amount_per_km_avg",
        "passenger_contribution_per_km_incentived_trips_avg",
        "driver_revenue_per_km_incentived_trips_avg",
    ],
    "week",
    (
        "Montants moyens <b>par km</b> des incitations,"
        "contributions passagers et revenus conducteurs"
        "<br><sub>Uniquement les trajets incités</sub>"
    ),
    {
        **labels_map,
        "incentive_amount_per_km_avg": "Incitation moyenne",
        "passenger_contribution_per_km_incentived_trips_avg": "Contribution moyenne passager",
        "driver_revenue_per_km_incentived_trips_avg": "Revenu moyen conducteur",
    },
    x_title="Montant (euros/km)",
)
fig_prices_per_km_by_week.show()


fig_prices_per_km_by_week.write_html(OUTPUT_PATH / "fig_prix_par_km_par_semaine.html")
fig_prices_per_km_by_week.write_image(
    OUTPUT_PATH / "fig_prix_par_km_par_semaine.svg", width=1280, height=720
)

### Incitation par rapport à la distance


In [ ]:
px.scatter(
    df_journeys.filter(
        pl.col("amount_aom").is_not_null()
        & (pl.col("incentive_sirets") == [AOM_SIRET])
        & (pl.col("is_fully_inside_campaign_area").not_())
    )
    .with_columns(pl.col("distance") / 1000)
    .unpivot(
        on=["amount_aom", "passenger_contribution", "driver_revenue"],
        index=["_id", "distance"],
    )
    .sort("distance"),
    x="distance",
    y="value",
    color="variable",
    template="simple_white",
)

In [ ]:
df_incentive_aom_by_distance = (
    df_journeys.filter(
        (pl.col("incentive_sirets") == [AOM_SIRET])
        & (pl.col("is_fully_inside_campaign_area"))
    )
    .with_columns(
        pl.col("distance") / 1000,
        pl.col("driver_revenue")
        .cum_sum()
        .over(
            partition_by=[
                "driver_identity_key",
                pl.col("start_datetime").dt.truncate("1mo"),
            ],
            order_by="start_datetime",
        )
        .alias("driver_revenue_cumsum"),
    )
    .filter(pl.col("driver_revenue_cumsum") <= 5000)
    .group_by(
        [
            pl.col("distance").cut(
                list(range(0, 100, 5)), include_breaks=True, left_closed=True
            ),
        ]
    )
    .agg(
        pl.len(),
        (pl.col("amount_aom") / 100).mean().alias("Incitation AOM"),
        (pl.col("passenger_contribution") / 100).alias("Contribution passager").mean(),
        (pl.col("driver_revenue") / 100).alias("Revenu conducteur").mean(),
    )
    .with_columns(pl.col("distance").struct.unnest())
    .unpivot(
        on=["Incitation AOM", "Contribution passager", "Revenu conducteur"],
        index=[
            "breakpoint",
            "category",
        ],
    )
    .sort(["breakpoint"])
)

In [ ]:
fig_incentive_aom_by_distance = px.line(
    df_incentive_aom_by_distance,
    x="breakpoint",
    y="value",
    color="variable",
    template="simple_white",
    height=800,
    labels={**labels_map, "breakpoint": "Distance", "value": "Montant (€)"},
)

fig_incentive_aom_by_distance.update_layout(
    legend_title="",
    title="Montants moyens du revenu conducteur, contribution passager et incitation AOM en fonction de la distance"
    "<br><sub>Uniquement les trajets inter, intervalles de distance de 5km.</sub>",
)
fig_incentive_aom_by_distance.show()

fig_incentive_aom_by_distance.write_html(
    "outputs_idfm/fig_incitation_aom_par_distance.html"
)
fig_incentive_aom_by_distance.write_image(
    "outputs_idfm/fig_incitation_aom_par_distance.svg", width=1280, height=720
)

# Conducteurs


In [ ]:
print(
    f"Nombre de conducteurs uniques : {
        df_journeys_raw.select(pl.col('driver_identity_key').n_unique()).item()
    }"
)

In [ ]:
fig_new_drivers_count_by_week = px.bar(
    df_journeys_raw.filter(
        pl.col("first_trip_datetime") >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT"))
    )
    .group_by(pl.col("first_trip_datetime").dt.truncate("1w").alias("week"))
    .agg(pl.len())
    .sort(pl.col("week")),
    x="week",
    y="len",
    labels={**labels_map, "len": "Nombre de nouveaux conducteurs"},
    template="simple_white",
    title="Evolution de l'acquisition des conducteurs",
)
fig_new_drivers_count_by_week.update_yaxes(showgrid=True)

fig_new_drivers_count_by_week.show()

fig_new_drivers_count_by_week.write_html(
    OUTPUT_PATH / "fig_conducteurs_par_semaine.html"
)
fig_new_drivers_count_by_week.write_image(
    OUTPUT_PATH / "fig_conducteurs_par_semaine.svg", width=1280, height=720
)

## Nombre de trajets


In [ ]:
df_journeys_raw.filter(
    pl.col("first_trip_datetime") >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT")),
    pl.col("first_trip_datetime") <= datetime.now(ZoneInfo("GMT")) - timedelta(days=14),
).group_by(["driver_identity_key", pl.col("start_datetime").dt.truncate("1w")]).agg(
    pl.len().alias("num_journeys"),
    pl.concat_str(pl.col("operator_id"), pl.lit("-"), pl.col("operator_trip_id"))
    .n_unique()
    .alias("num_trips"),
).group_by(["start_datetime"]).agg(
    pl.col("num_journeys").mean().alias("Nombre moyen de journeys par semaine"),
    pl.col("num_trips").mean().alias("Nombre moyen de trips par semaine"),
).select(
    pl.col("Nombre moyen de journeys par semaine").mean(),
    pl.col("Nombre moyen de trips par semaine").mean(),
)

In [ ]:
df_journeys.filter(
    pl.col("first_trip_datetime") >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT")),
    pl.col("first_trip_datetime") <= datetime.now(ZoneInfo("GMT")) - timedelta(days=30),
    pl.col("start_datetime") <= pl.col("first_trip_datetime") + pl.duration(days=30),
).group_by(["driver_identity_key"]).agg(
    pl.len().alias("num_journeys"),
    pl.concat_str(pl.col("operator_id"), pl.lit("-"), pl.col("operator_trip_id"))
    .n_unique()
    .alias("num_trips"),
).select(
    pl.col("num_journeys").mean().alias("Nombre moyen de journeys sur 30 jours"),
    pl.col("num_trips").mean().alias("Nombre moyen de trips sur 30 jours"),
)

In [ ]:
breaks = range(1, 31, 2)
data_agg = (
    df_journeys_raw.filter(
        pl.col("first_trip_datetime") >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT")),
        pl.col("first_trip_datetime")
        <= datetime.now(ZoneInfo("GMT")) - timedelta(days=30),
        pl.col("start_datetime")
        <= pl.col("first_trip_datetime") + pl.duration(days=30),
    )
    .group_by(["driver_identity_key"])
    .agg(
        pl.len().alias("num_journeys"),
        pl.concat_str(pl.col("operator_id"), pl.lit("-"), pl.col("operator_trip_id"))
        .n_unique()
        .alias("num_trips"),
    )
    .with_columns(
        pl.col("num_trips").cut(breaks=breaks, left_closed=True, include_breaks=True)
    )
    .group_by(["num_trips"])
    .agg(pl.col("driver_identity_key").n_unique().alias("num_drivers"))
    .with_columns(
        pl.col("num_trips").struct.unnest(),
        (100 * pl.col("num_drivers") / pl.col("num_drivers").sum())
        .round(2)
        .alias("share_drivers"),
    )
    .sort("breakpoint")
)

fig_drivers_by_trip_numbers_hist = px.bar(
    data_agg,
    x=data_agg["category"],
    y=data_agg["share_drivers"],
    labels={**labels_map},
    text="share_drivers",
    text_auto=".1f",
    template="simple_white",
    title="Distribution du nombre de conducteurs en fonction du nombre de trajets effectués <br><sub>Période de 30 jour</sub>",
)

fig_drivers_by_trip_numbers_hist.update_yaxes(showgrid=True)
fig_drivers_by_trip_numbers_hist.show()

fig_drivers_by_trip_numbers_hist.write_html(
    OUTPUT_PATH / "fig_histo_trajets_conducteurs.html"
)
fig_drivers_by_trip_numbers_hist.write_image(
    OUTPUT_PATH / "fig_histo_trajets_conducteurs.svg", width=1280, height=720
)

## Rétention


In [ ]:
NUM_WEEKS = 3 * 4
df_acquisition_by_driver_type = (
    (
        df_journeys.filter(
            pl.col("first_trip_datetime")
            >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT")),
            pl.col("first_trip_datetime")
            <= datetime.now(ZoneInfo("GMT")) - timedelta(weeks=NUM_WEEKS),
        )
        .group_by(["driver_identity_key"])
        .agg(
            pl.col("start_datetime").min(),
            pl.datetime_range(
                pl.col("start_datetime").min().dt.truncate("1w"),
                pl.col("start_datetime").min().dt.truncate("1w")
                + pl.duration(weeks=NUM_WEEKS),
                "1w",
            ).alias("week"),
        )
        .explode("week")
        .join(
            df_journeys.filter(
                pl.col("start_datetime")
                >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT")),
            ),
            left_on=["driver_identity_key", "week"],
            right_on=[
                "driver_identity_key",
                pl.col("start_datetime").dt.truncate("1w"),
            ],
            how="left",
        )
    )
    .group_by([pl.col("driver_identity_key"), "week"])
    .agg(
        (pl.col("_id").count() > 0).alias("has_traveled"),
    )
    .with_columns(
        pl.col("week")
        .rank()
        .over(partition_by=["driver_identity_key"], order_by="week")
        .alias("week_number")
    )
    .group_by(["week_number"])
    .agg(
        (100 * pl.col("has_traveled").sum() / pl.col("has_traveled").count()).alias(
            "drivers_share"
        )
    )
    .sort([pl.col("week_number")])
)
df_acquisition_by_driver_type

In [ ]:
fig_churn_by_campaign_type = px.line(
    df_acquisition_by_driver_type,
    x="week_number",
    y="drivers_share",
    template="simple_white",
    labels=labels_map,
    title="Attrition des conducteurs - Période de 12 semaines",
)

fig_churn_by_campaign_type.update_yaxes(showgrid=True)
fig_churn_by_campaign_type.show()

fig_churn_by_campaign_type.write_html(OUTPUT_PATH / "fig_attrition_par_campagne.html")
fig_churn_by_campaign_type.write_image(
    OUTPUT_PATH / "fig_attrition_par_campagne.svg", width=1280, height=720
)

# Passagers


In [ ]:
print(
    f"Nombre de passagers uniques : {
        df_journeys_raw.select(pl.col('passenger_identity_key').n_unique()).item()
    }"
)

## Pass Navigo


In [ ]:
df_passengers_navigo = df_journeys_raw.group_by("passenger_identity_key").agg(
    pl.col("passenger_travelpass_name").max(),
    pl.col("passenger_travelpass_user_id").max(),
)
df_passengers_navigo

In [ ]:
f"Nombre de passagers avec un pass navigo : {
    df_passengers_navigo.select(
        pl.col('passenger_travelpass_user_id').is_not_null().sum()
    ).item()
}"

**NOTE** : J'ai identifié que il n'y a pas de trajets avec le pass navigo renseigné depuis septembre 2020 si on exclut 9 trajets en 2025


# Autour des gares


## Transformations spatiales


In [ ]:
%%sql
FROM
  (DESCRIBE gares_idfm)

In [ ]:
conn.sql(
    """
create or replace table journeys_raw_with_stations_start as (
SELECT
      jr."_id",
      jr.start_pos,
      jr.end_pos,
      g.id_gares as id_gares_start,
      g.nom_gares as nom_gares_start,
      g.mode as mode_start,
      g.geom as geom_start,
      ST_DISTANCE(jr.start_pos,g.geom) as distance_to_station_start,
      ST_DISTANCE_SPHERE(jr.start_pos,g.geom) as distance_to_station_sphere_start,
      ST_Distance_Spheroid(jr.start_pos,g.geom) as distance_to_station_spheroid_start
  FROM journeys_raw jr
  left join gares_idfm g on ST_DWithin(jr.start_pos,g.geom,0.1)
)

    """
)

In [ ]:
conn.sql(
    """
create or replace table journeys_raw_with_nearest_stations_start as (
SELECT
      *
FROM journeys_raw_with_stations_start
qualify (row_number() over (partition by "_id" order by distance_to_station_sphere_start asc nulls last))=1
)
    """
)

In [ ]:
%%sql
DROP TABLE journeys_raw_with_stations_start

In [ ]:
%sql CHECKPOINT

In [ ]:
%%sql
SELECT
  COUNT(*)
FROM
  journeys_raw_with_nearest_stations_start

In [ ]:
%%sql
FROM
  (DESCRIBE journeys_raw_with_nearest_stations_start)

In [ ]:
conn.sql(
    """
create or replace table journeys_raw_with_stations_end as (
SELECT
      jr.*,
      g.id_gares as id_gares_end,
      g.nom_gares as nom_gares_end,
      g.mode as mode_end,
      g.geom as geom_end,
      ST_DISTANCE(jr.end_pos,g.geom) as distance_to_station_end,
      ST_DISTANCE_SPHERE(jr.end_pos,g.geom) as distance_to_station_sphere_end,
      ST_Distance_Spheroid(jr.end_pos,g.geom) as distance_to_station_spheroid_end
  FROM journeys_raw_with_nearest_stations_start jr
  left join gares_idfm g on ST_DWithin(jr.end_pos,g.geom,0.1)
)
"""
)

In [ ]:
%%sql
SELECT
  COUNT(*)
FROM
  journeys_raw_with_stations_end

In [ ]:
%%sql
DROP TABLE journeys_raw_with_nearest_stations_start

In [ ]:
%%sql
checkpoint

In [ ]:
conn.sql(
    """
create or replace table journeys_raw_with_nearest_stations as (
SELECT
      *
FROM journeys_raw_with_stations_end
qualify (row_number() over (partition by "_id" order by distance_to_station_sphere_end asc nulls last))=1
)
    """
)

In [ ]:
%%sql 
DROP TABLE journeys_raw_with_stations_end

In [ ]:
%%sql
SELECT
  COUNT(*)
FROM
  journeys_raw_with_nearest_stations

In [ ]:
%%sql
FROM
  (DESCRIBE journeys_raw_with_nearest_stations)

In [ ]:
%%sql df_journeys_raw_with_nearest_stations <<
SELECT
  *,
  ST_AsText(start_pos) as start_pos_wkt,
  ST_AsText(end_pos) as end_pos_wkt,
  ST_AsText (geom_start) AS geom_start_wkt,
  ST_AsText (geom_end) AS geom_end_wkt
FROM
  journeys_raw_with_nearest_stations

In [ ]:
df_journeys_raw_with_nearest_stations = (
    df_journeys_raw_with_nearest_stations.with_columns(
        pl.selectors.starts_with("distance_to_station").fill_null(float("+inf"))
    )
)

## Distribution des distances à la gare la plus proche


### Point de départ


In [ ]:
breaks = [100, 200, 500] + list(range(1000, 11000, 1000))

df_journeys_count_by_distance_to_nearest_station_start = (
    df_journeys_raw_with_nearest_stations.group_by(
        pl.col("distance_to_station_spheroid_start").cut(
            breaks=breaks, include_breaks=True
        )
    )
    .agg(pl.col("_id").n_unique().alias("num_journeys"))
    .with_columns(
        (pl.col("num_journeys") / pl.col("num_journeys").sum()).alias("share_journeys"),
        pl.col("distance_to_station_spheroid_start").struct.unnest(),
    )
    .with_columns(
        pl.format("{}%", (100 * pl.col("share_journeys")).round(2)).alias(
            "share_journeys_fmt"
        )
    )
    .sort("category")
)

In [ ]:
px.bar(
    df_journeys_count_by_distance_to_nearest_station_start,
    x="category",
    y="num_journeys",
    text="share_journeys_fmt",
    labels={**labels_map, "category": "Catégorie de distance (en mètres)"},
    template="simple_white",
    title="Distribution du nombre de trajets en fonction de la distance à la gare RER/Transilien la plus proche"
    "<br><sub>Par rapport au point de départ du trajet</sub>",
)

### Point d'arrivée


In [ ]:
df_journeys_count_by_distance_to_nearest_station_end = (
    df_journeys_raw_with_nearest_stations.group_by(
        pl.col("distance_to_station_spheroid_end").cut(
            breaks=[100, 200, 500] + list(range(1000, 11000, 1000)), include_breaks=True
        )
    )
    .agg(pl.col("_id").n_unique().alias("num_journeys"))
    .with_columns(
        (pl.col("num_journeys") / pl.col("num_journeys").sum()).alias("share_journeys"),
        pl.col("distance_to_station_spheroid_end").struct.unnest(),
    )
    .with_columns(
        pl.format("{}%", (100 * pl.col("share_journeys")).round(2)).alias(
            "share_journeys_fmt"
        )
    )
    .sort("category")
)

In [ ]:
px.bar(
    df_journeys_count_by_distance_to_nearest_station_end,
    x="category",
    y="num_journeys",
    text="share_journeys_fmt",
    labels={**labels_map, "category": "Catégorie de distance (en mètres)"},
    template="simple_white",
    title="Distribution du nombre de trajets en fonction de la distance à la gare RER/Transilien la plus proche"
    "<br><sub>Par rapport au point d'arrivée du trajet</sub>",
)

## Origine et destination


In [ ]:
breaks = [100, 200, 500] + list(range(1000, 11000, 1000))
df_journeys_count_by_distance_to_nearest_station = (
    df_journeys_raw_with_nearest_stations.with_columns(
        pl.max_horizontal(
            ["distance_to_station_spheroid_start", "distance_to_station_spheroid_end"]
        ).alias("max_distance_to_nearest_station")
    )
    .group_by(
        pl.col("max_distance_to_nearest_station").cut(
            breaks=[100, 200, 500] + list(range(1000, 11000, 1000)), include_breaks=True
        )
    )
    .agg(pl.col("_id").n_unique().alias("num_journeys"))
    .with_columns(
        (pl.col("num_journeys") / pl.col("num_journeys").sum()).alias("share_journeys"),
        pl.col("max_distance_to_nearest_station").struct.unnest(),
    )
    .with_columns(
        pl.format("{}%", (100 * pl.col("share_journeys")).round(2)).alias(
            "share_journeys_fmt"
        )
    )
    .sort("category")
)

In [ ]:
px.bar(
    df_journeys_count_by_distance_to_nearest_station,
    x="category",
    y="num_journeys",
    text="share_journeys_fmt",
    labels={**labels_map, "category": "Catégorie de distance (en mètres)"},
    template="simple_white",
    title="Distribution du nombre de trajets en fonction de la distance à la gare RER/Transilien la plus proche"
    "<br><sub>Par rapport au point de départ ET d'arrivée du trajet</sub>",
)

### Multi point


In [ ]:
df_journeys_count_by_distance_to_nearest_stations = (
    df_journeys_raw_with_nearest_stations.group_by(
        pl.col("distance_to_station_spheroid_start").cut(
            breaks=[100, 200, 500] + list(range(1000, 11000, 1000)), include_breaks=True
        )
    )
    .agg(pl.col("_id").n_unique().alias("num_journeys"))
    .with_columns(
        (pl.col("num_journeys") / pl.col("num_journeys").sum()).alias("share_journeys"),
        pl.col("distance_to_station_spheroid_start").struct.unnest(),
    )
    .with_columns(
        pl.format("{}%", (100 * pl.col("share_journeys")).round(2)).alias(
            "share_journeys_fmt"
        )
    )
    .sort("category")
)

In [ ]:
px.scatter(
    df_journeys_raw_with_nearest_stations.group_by(
        (
            pl.col("distance_to_station_spheroid_start").fill_null(float("inf")) / 100
        ).round(0),
        (
            pl.col("distance_to_station_spheroid_end").fill_null(float("inf")) / 100
        ).round(0),
    ).agg(pl.col("_id").n_unique().alias("num_journeys")),
    x="distance_to_station_spheroid_start",
    y="distance_to_station_spheroid_end",
    size="num_journeys",
    template="simple_white",
)

## Top des gares


### Au point de départ


In [ ]:
df_top_nearest_stations_start = (
    df_journeys_raw_with_nearest_stations.filter(
        pl.col("distance_to_station_spheroid_start").is_finite()
    )
    .group_by("nom_gares_start")
    .agg(
        pl.col("_id").n_unique().alias("num_journeys"),
        pl.col("geom_start_wkt").max(),
        pl.col("distance_to_station_spheroid_start").mean(),
    )
    .with_columns(
        (pl.col("num_journeys") / pl.col("num_journeys").sum()).alias("share_journeys")
    )
    .with_columns(
        pl.format(
            "{}%<br>{}m",
            (100 * pl.col("share_journeys")).round(2),
            pl.col("distance_to_station_spheroid_start").round(0).cast(pl.Int64),
        ).alias("share_journeys_fmt")
    )
    .sort("num_journeys", descending=True)
    .drop_nulls()
)

In [ ]:
px.bar(
    df_top_nearest_stations_start.head(10),
    x="nom_gares_start",
    y="num_journeys",
    text="share_journeys_fmt",
    template="simple_white",
    labels=labels_map,
    title="TOP 10 des gares les plus proches des points de départ"
    "<br><sub>Dans les barres sont affichées la proportion au regard du total des trajets et la distance moyenne au départ.</sub>",
    height=500,
)

#### Carto


In [ ]:
df_top_nearest_stations_start = df_top_nearest_stations_start.with_columns(
    pl.col("geom_start_wkt").map_elements(shapely.from_wkt, return_dtype=pl.Object)
)

In [ ]:
# Method 2: More robust approach using shapely (if you have it installed)
def create_top_k_nearest_station_map(df: pl.DataFrame, top_k: int = 10):
    df = df.head(top_k)
    # Create folium map centered on data
    center_lat = 48.866667
    center_lon = 2.333333

    m = folium.Map(location=[center_lat, center_lon], zoom_start=9)

    # Process each row
    for row in df.iter_rows(named=True):
        point = row["geom_start_wkt"]  # geometry column
        num_journeys = row["num_journeys"]  # num_journeys column
        station_name = row["nom_gares_start"]
        try:
            # Extract coordinates
            lon, lat = point.y, point.x
            text = f"{station_name},<br>{num_journeys} Journeys"
            # Create bubble with size proportional to num_journeys
            folium.CircleMarker(
                location=[lat, lon],
                radius=max(10, num_journeys / 2000),  # Adjust scaling factor
                popup=text,
                tooltip=text,
                fill=True,
                fill_color="cornflowerblue",
                fill_opacity=0.7,
                stroke=False,
            ).add_to(m)

        except Exception as e:
            print(f"Error processing point: {e}")
            continue

    return m

In [ ]:
m = create_top_k_nearest_station_map(df_top_nearest_stations_start, 50)
m

## À moins d'un kilometre


In [ ]:
df_top_nearest_stations_start_1km = (
    df_journeys_raw_with_nearest_stations.filter(
        pl.col("distance_to_station_spheroid_start") <= 1000
    )
    .group_by("nom_gares_start")
    .agg(
        pl.col("_id").n_unique().alias("num_journeys"),
        pl.col("geom_start_wkt").max(),
        pl.col("distance_to_station_spheroid_start").mean(),
    )
    .with_columns(
        (pl.col("num_journeys") / pl.col("num_journeys").sum()).alias("share_journeys")
    )
    .with_columns(
        pl.format(
            "{}%<br>{}m",
            (100 * pl.col("share_journeys")).round(2),
            pl.col("distance_to_station_spheroid_start").round(0).cast(pl.Int64),
        ).alias("share_journeys_fmt")
    )
    .sort("num_journeys", descending=True)
    .drop_nulls()
)

In [ ]:
px.bar(
    df_top_nearest_stations_start_1km.head(10),
    x="nom_gares_start",
    y="num_journeys",
    text="share_journeys_fmt",
    template="simple_white",
    labels=labels_map,
    title="TOP 10 des gares dans un rayon d'un kilomètre des points de départ"
    "<br><sub>Dans les barres sont affichées la proportion au regard du total des trajets et la distance moyenne au départ.</sub>",
    height=500,
)

## heatmap


In [ ]:
def create_trip_heatmap_map(
    df_journeys_raw_with_nearest_stations, stations_df, distance_filter: int = 1000
):
    # Create a folium map centered on Paris
    m = folium.Map(
        location=[48.8566, 2.3522],  # Paris coordinates
        zoom_start=9,
        tiles="Cartodb Positron",
        control_scale=True,
    )

    # Prepare trip data for heatmap
    trips_data = []
    for row in df_journeys_raw_with_nearest_stations.filter(
        pl.max_horizontal(
            pl.col("distance_to_station_spheroid_start"),
            pl.col("distance_to_station_spheroid_end"),
        )
        <= distance_filter
    ).iter_rows(named=True):
        start_point = shapely.from_wkt(row["start_pos_wkt"])
        start_lat, start_lon = (
            start_point.x,
            start_point.y,
        )
        trips_data.append([start_lat, start_lon])

    # Create heatmap layer
    if trips_data:
        heat_data = [[point[0], point[1]] for point in trips_data]
        plugins.HeatMap(heat_data, min_opacity=0.4, radius=15, blur=20).add_to(m)

    # Add station points
    for row in stations_df.iter_rows(named=True):
        point = shapely.from_wkt(row["geom_wkt"])
        station_lat, station_lon = point.x, point.y
        folium.CircleMarker(
            location=[station_lat, station_lon],
            radius=2,
            color="black",
            fillOpacity=0.7,
            tooltip=row["nom_gares"],
        ).add_to(m)

    #    paris_wkt_str = ""
    #    with open("paris_wkt.txt", "r") as f:
    #        paris_wkt_str = f.read()
    #    paris_polygon = shapely.from_wkt(paris_wkt_str).exterior

    # Create polygon layer
    #   folium.Polygon(
    #       locations=[[y, x] for x, y in list(paris_polygon.coords)],
    #       color="white",
    #       weight=3,
    #       fill=False,
    #       tooltip="Paris",
    #   ).add_to(m)

    return m

In [ ]:
m = create_trip_heatmap_map(df_journeys_raw_with_nearest_stations, df_idfm_stations)
m

## Analyse des trajets avec O et D à proximité d'une gare


In [ ]:
MAX_DISTANCE = 1000  # 1 km

In [ ]:
df_journeys_raw_with_nearest_stations_full = (
    df_journeys_raw.join(
        df_journeys_raw_with_nearest_stations.filter(
            pl.max_horizontal(
                "distance_to_station_spheroid_start", "distance_to_station_spheroid_end"
            )
            <= MAX_DISTANCE
        ),
        on="_id",
        how="left",
        validate="1:1",
        coalesce=False,
    )
    .with_columns(pl.col("_id_right").is_not_null().alias("is_near_station"))
    .with_columns(
        pl.when(pl.col("is_near_station"))
        .then(pl.lit("Trajets proches d'une gare"))
        .otherwise(pl.lit("Trajets éloignés d'une gare"))
        .alias("is_near_station_fmt")
    )
)
df_journeys_raw_with_nearest_stations_full.shape

In [ ]:
with pl.Config(set_fmt_str_lengths=120, set_tbl_width_chars=1000):
    print(
        df_journeys_raw_with_nearest_stations_full.filter(pl.col("is_near_station"))
        .select(
            pl.col("_id").n_unique().alias("Nombre de journeys"),
            pl.col("_id")
            .filter(pl.col("incentive_amount") > 0)
            .n_unique()
            .alias("Nombre de journeys avec incitation"),
            (
                100
                * pl.col("_id").filter(pl.col("incentive_amount") > 0).n_unique()
                / pl.col("_id").n_unique()
            ).alias("% journeys avec incitation"),
            pl.col("_id")
            .filter(pl.col("incentived_by_aom"))
            .n_unique()
            .alias("Nombre de journeys avec incitation AOM"),
            (
                100
                * pl.col("_id").filter(pl.col("incentived_by_aom")).n_unique()
                / pl.col("_id").n_unique()
            ).alias("% journeys avec incitation AOM"),
            pl.col("_id")
            .filter(pl.col("incentived_by_operator"))
            .n_unique()
            .alias("Nombre de journeys avec incitation opérateur"),
            (
                100
                * pl.col("_id").filter(pl.col("incentived_by_operator")).n_unique()
                / pl.col("_id").n_unique()
            ).alias("% journeys avec incitation opérateur"),
        )
        .with_columns(pl.selectors.all().round(2))
        .unpivot()
    )

In [ ]:
df_stats_by_month_full = (
    df_journeys_raw_with_nearest_stations_full.group_by(
        [
            pl.col("start_datetime").dt.truncate("1mo").alias("month"),
            "is_near_station_fmt",
        ]
    )
    .agg(agg_expressions)
    .sort(pl.col("month"))
)

In [ ]:
df_stats_by_week_filtered = (
    df_journeys_raw_with_nearest_stations_full.filter(
        pl.col("start_datetime") <= datetime(2025, 7, 20, tzinfo=ZoneInfo("GMT"))
    )
    .group_by(
        [
            pl.col("start_datetime").dt.truncate("1w").alias("week"),
            "is_near_station_fmt",
        ]
    )
    .agg(agg_expressions)
    .sort(pl.col("week"))
)

### Evolution


#### Globale


In [ ]:
fig_journeys_by_month_near_vs_far = px.line(
    df_stats_by_month_full,
    x="month",
    y="num_journeys_incentived",
    color="is_near_station_fmt",
    color_discrete_map=colors_map,
    template="simple_white",
    labels=labels_map,
    title="Nombre de journeys par mois - "
    f'<br><sub>Les trajets dits "proches" sont ceux avec O/D à moins de {MAX_DISTANCE / 1000:.0f} km d\'une gare<sub>',
)
fig_journeys_by_month_near_vs_far.show()

fig_journeys_by_month_near_vs_far
fig_journeys_by_month_near_vs_far.update_yaxes(
    range=[0, df_stats_by_month_full["num_journeys_incentived"].max() - 1.1]
)

fig_journeys_by_month_near_vs_far.write_html(
    OUTPUT_PATH / "fig_journeys_par_mois_near_vs_far.html"
)
fig_journeys_by_month_near_vs_far.write_image(
    OUTPUT_PATH / "fig_journeys_par_mois_near_vs_far.svg", width=1280, height=720
)

#### Opérateur incitateurs


In [ ]:
fig_journeys_by_operator_filtered = px.line(
    df_journeys_raw_with_nearest_stations_full.filter(pl.col("is_near_station"))
    .explode("incentive_sirets")
    .join(
        df_operators,
        left_on="incentive_sirets",
        right_on="siret",
        how="left",
        suffix="_operators",
    )
    .group_by(["name", pl.col("start_datetime").dt.truncate("1mo")])
    .agg(pl.col("operator_journey_id").n_unique().alias("num_journeys"))
    .rename({"name": "operator", "start_datetime": "month"})
    .sort("month"),
    x="month",
    y="num_journeys",
    color="operator",
    template="simple_white",
    labels=labels_map,
    title="Nombre de journeys incités par opérateur"
    f"<br><sub>Uniquement les trajets avec O/D à moins de {MAX_DISTANCE / 1000:.0f} km d'une gare</sub>",
)
fig_journeys_by_operator_filtered.update_yaxes(showgrid=True)
fig_journeys_by_operator_filtered.show()


fig_journeys_by_operator_filtered.write_html(
    "outputs_idfm/fig_journeys_par_operateur_mois_filtered.html"
)
fig_journeys_by_operator_filtered.write_image(
    "outputs_idfm/fig_journeys_par_operateur_mois_filtered.svg", width=1280, height=720
)

## Par opérateurs


In [ ]:
px.line(
    (
        df_journeys_raw_with_nearest_stations_full.filter(
            incentived_trip_filter_expr & pl.col("is_near_station")
        )
        .group_by(["operator_id", pl.col("start_datetime").dt.truncate("1w")])
        .agg(pl.col("_id").n_unique().alias("num_journeys"))
        .join(
            df_operators,
            left_on="operator_id",
            right_on="_id",
            validate="m:1",
            suffix="_operators",
        )
        .sort(["start_datetime", "operator_id"])
    ),
    title="Nombre de journeys par opérateur"
    f"<br><sub>Uniquement les trajets avec O/D à moins de {MAX_DISTANCE / 1000:.0f} km d'une gare</sub>",
    x="start_datetime",
    y="num_journeys",
    color="name",
    labels=labels_map,
    template="simple_white",
)

# Comparaisons entre les trajets proches et éloignés d'une gare


In [ ]:
colors_map = {
    "Trajets éloignés d'une gare": "#ffae2b",
    "Trajets proches d'une gare": "#2B7CFF",
}
category_order = ["Trajets proches d'une gare", "Trajets éloignés d'une gare"]

## Distance


In [ ]:
fig_distance_by_month_near_vs_far = px.line(
    df_stats_by_month_full.with_columns(
        pl.col("distance_avg").round(1).alias("distance_avg_fmt")
    ),
    x="month",
    y="distance_avg",
    text="distance_avg_fmt",
    color="is_near_station_fmt",
    color_discrete_map=colors_map,
    category_orders={"is_near_station_fmt": category_order},
    template="simple_white",
    labels=labels_map,
    title="Distance parcourue moyenne par journeys avec incentives par mois - "
    f'<br><sub>Les trajets dits "proches" sont ceux avec O/D à moins de {MAX_DISTANCE / 1000:.0f} km d\'une gare<sub>',
)
fig_distance_by_month_near_vs_far.show()

fig_distance_by_month_near_vs_far
fig_distance_by_month_near_vs_far.update_traces(textposition="top center")

fig_distance_by_month_near_vs_far.update_yaxes(
    range=[0, df_stats_by_month_full["distance_avg"].max() - 1.1]
)

fig_distance_by_month_near_vs_far.write_html(
    OUTPUT_PATH / "fig_distance_avg_by_month_near_vs_far.html"
)
fig_distance_by_month_near_vs_far.write_image(
    OUTPUT_PATH / "fig_distance_avg_by_month_near_vs_far.svg", width=1280, height=720
)

Les trajets qui débutent et finissent proches d'une gare sont en moyennes plus courts que les autres trajets. Cela pourrait par exemple traduire un déplacement hybride (voiture + transports en communs). S'ajoute à cela le fait que les gares soient des points de rencontre. A priori, la plupart des trajets se dirigent vers Paris - les points d'arrivées sont peut etre des gares qui connectent rapidement au métro/permettent de se rendre rapidement dans Paris. Les gares sont peut-être à proximité de parking relais.
TODO: vérifier les hypothèses pertinentes, trouver d'autres hypothèses.


## Par opérateurs


In [ ]:
px.line(
    (
        df_journeys_raw_with_nearest_stations_full.filter(incentived_trip_filter_expr)
        .group_by(
            [
                "operator_id",
                pl.col("start_datetime").dt.truncate("1w"),
                "is_near_station_fmt",
            ]
        )
        .agg(pl.col("_id").n_unique().alias("num_journeys"))
        .with_columns(
            (
                pl.col("num_journeys")
                / pl.col("num_journeys")
                .sum()
                .over(["start_datetime", "is_near_station_fmt"])
                * 100
            ).alias("percentage_journeys")
        )
        .join(
            df_operators,
            left_on="operator_id",
            right_on="_id",
            validate="m:1",
            suffix="_operators",
        )
        .sort(["start_datetime", "operator_id"])
    ),
    title=f"Proportion de journeys par opérateur par catégorie de distance à une gare <br><sub>Les trajets dits proches sont ceux avec O/D à moins de {MAX_DISTANCE / 1000:.0f} km d'une gare</sub>",
    x="start_datetime",
    y="percentage_journeys",
    color="name",
    line_dash="is_near_station_fmt",
    labels=labels_map,
    template="simple_white",
)

Les répartitions par opérateurs sont les mêmes que le trajet soit proche d\'une gare ou non. Certains opérateurs sont utilisés uniquement pour les trajets éloignés des gares - surement du à un faible échantillon


## Prix, revenus et incitations


### Trajets incités


In [ ]:
df_stats_by_week_near = (
    df_journeys_raw_with_nearest_stations_full.filter(
        incentived_trip_filter_expr & pl.col("is_near_station")
    )
    .group_by([pl.col("start_datetime").dt.truncate("1w").alias("week")])
    .agg(agg_expressions)
    .sort("week")
)

df_stats_by_week_far = (
    df_journeys_raw_with_nearest_stations_full.filter(
        incentived_trip_filter_expr & ~pl.col("is_near_station")
    )
    .group_by([pl.col("start_datetime").dt.truncate("1w").alias("week")])
    .agg(agg_expressions)
    .sort("week")
)

#### Proche d'une gare


In [ ]:
fig_prices_by_week_near = create_scatter_fig_prices(
    df_stats_by_week_near,
    [
        "incentive_amount_avg",
        "passenger_contribution_incentived_trips_avg",
        "driver_revenue_incentived_trips_avg",
    ],
    "week",
    (
        "Montants moyens par trajet des incitations pour les trajets <b>proches d'une gare</b>,"
        "<br>contributions passagers et revenus conducteurs - Trajets incités"
    ),
    {
        **labels_map,
        "passenger_contribution_incentived_trips_avg": "Contribution moyenne passager",
        "driver_revenue_incentived_trips_avg": "Revenu moyen conducteur",
    },
)
fig_prices_by_week_near.show()


fig_prices_by_week_near.write_html(OUTPUT_PATH / "fig_prix_par_semaine_near.html")
# fig_prices_by_week_near.write_image(
#    OUTPUT_PATH / "fig_prix_par_semaine_near.svg", width=1280, height=720
# )

#### Eloignés d'une gare


In [ ]:
fig_prices_by_week_far = create_scatter_fig_prices(
    df_stats_by_week_far,
    [
        "incentive_amount_avg",
        "passenger_contribution_incentived_trips_avg",
        "driver_revenue_incentived_trips_avg",
    ],
    "week",
    (
        "Montants moyens par trajet des incitations pour les trajets <b>éloignés d'une gare</b>,"
        "<br>contributions passagers et revenus conducteurs - Trajets incités"
    ),
    {
        **labels_map,
        "passenger_contribution_incentived_trips_avg": "Contribution moyenne passager",
        "driver_revenue_incentived_trips_avg": "Revenu moyen conducteur",
    },
)
fig_prices_by_week_far.show()


fig_prices_by_week_far.write_html(OUTPUT_PATH / "fig_prix_par_semaine_far.html")
# fig_prices_by_week_far.write_image(
#    OUTPUT_PATH / "fig_prix_par_semaine_far.svg", width=1280, height=720
# )

#### Intra


##### Proches d'une gare


In [ ]:
fig_prices_by_week_intra_near = create_scatter_fig_prices(
    df_stats_by_week_near,
    [
        "incentive_amount_intra_avg",
        "passenger_contribution_intra_avg",
        "driver_revenue_intra_avg",
    ],
    "week",
    (
        "Montants moyens par trajet <b>intra proche d'une gare</b>  des incitations,"
        "<br>contributions passagers et revenus conducteurs"
    ),
    labels_map,
)
fig_prices_by_week_intra_near.show()


fig_prices_by_week_intra_near.write_html(
    OUTPUT_PATH / "fig_prix_intra_par_semaine_near.html"
)
# fig_prices_by_week_intra_near.write_image(
#    OUTPUT_PATH / "fig_prix_intra_par_semaine_near.svg", width=1280, height=720
#

##### Eloignés d'une gare


In [ ]:
fig_prices_by_week_intra_far = create_scatter_fig_prices(
    df_stats_by_week_far,
    [
        "incentive_amount_intra_avg",
        "passenger_contribution_intra_avg",
        "driver_revenue_intra_avg",
    ],
    "week",
    (
        "Montants moyens par trajet <b>intra éloigné d'une gare</b> des incitations,"
        "<br>contributions passagers et revenus conducteurs"
    ),
    labels_map,
)
fig_prices_by_week_intra_far.show()


fig_prices_by_week_intra_far.write_html(
    OUTPUT_PATH / "fig_prix_intra_par_semaine_far.html"
)
# fig_prices_by_week_intra_far.write_image(
#    OUTPUT_PATH / "fig_prix_intra_par_semaine_near.svg", width=1280, height=720
#

#### Inter


##### Proches d'une gare


In [ ]:
fig_prices_by_week_inter_near = create_scatter_fig_prices(
    df_stats_by_week_near,
    [
        "incentive_amount_inter_avg",
        "passenger_contribution_inter_avg",
        "driver_revenue_inter_avg",
    ],
    "week",
    (
        "Montants moyens par trajet <b>inter proche d'une gare</b>  des incitations,"
        "<br>contributions passagers et revenus conducteurs"
    ),
    labels_map,
)
fig_prices_by_week_inter_near.show()


fig_prices_by_week_inter_near.write_html(
    OUTPUT_PATH / "fig_prix_inter_par_semaine_near.html"
)
# fig_prices_by_week_inter_near.write_image(
#    OUTPUT_PATH / "fig_prix_inter_par_semaine_near.svg", width=1280, height=720
#

##### Eloignés d'une gare


In [ ]:
fig_prices_by_week_inter_far = create_scatter_fig_prices(
    df_stats_by_week_far,
    [
        "incentive_amount_inter_avg",
        "passenger_contribution_inter_avg",
        "driver_revenue_inter_avg",
    ],
    "week",
    (
        "Montants moyens par trajet <b>inter éloigné d'une gare</b> des incitations,"
        "<br>contributions passagers et revenus conducteurs"
    ),
    labels_map,
)
fig_prices_by_week_inter_far.show()


fig_prices_by_week_inter_far.write_html(
    OUTPUT_PATH / "fig_prix_inter_par_semaine_far.html"
)
# fig_prices_by_week_inter_far.write_image(
#    OUTPUT_PATH / "fig_prix_inter_par_semaine_near.svg", width=1280, height=720
#

## Nombre de passagers


In [ ]:
df_stats_with_nearest_stations_full = (
    df_journeys_raw_with_nearest_stations_full.filter(incentived_trip_filter_expr)
    .group_by(pl.col("is_near_station"))
    .agg(agg_expressions)
)
df_stats_with_nearest_stations_full.select(
    pl.col("number_of_unique_passenger").sum().alias("Nombre de passagers"),
    pl.col("number_of_unique_passenger")
    .filter(
        pl.col("is_near_station"),
    )
    .alias("Nombre de passagers effectuants des trajets proches de gare"),
    pl.col("number_of_unique_passenger")
    .filter(
        pl.col("is_near_station").not_(),
    )
    .alias("Nombre de passagers effectuants des trajets éloignés de gare"),
)

In [ ]:
print(df_journeys_raw_with_nearest_stations_full.columns)

In [ ]:
px.line(
    (
        df_journeys_raw_with_nearest_stations_full.filter(incentived_trip_filter_expr)
        .group_by([pl.col("start_datetime").dt.truncate("1w"), "is_near_station_fmt"])
        .agg(pl.col("passenger_identity_key").n_unique().alias("num_passenger"))
        .sort(["start_datetime"])
    ),
    title=f"Nombre de passager par catégorie de distance à une gare <br><sub>Les trajets dits proches sont ceux avec O/D à moins de {MAX_DISTANCE / 1000:.0f} km d'une gare</sub>",
    x="start_datetime",
    y="num_passenger",
    color="is_near_station_fmt",
    color_discrete_map=colors_map,
    category_orders={"is_near_station_fmt": category_order},
    labels=labels_map,
    template="simple_white",
)

## Conducteurs


### Acquisition


In [ ]:
fig_new_drivers_count_by_week = px.bar(
    df_journeys_raw_with_nearest_stations_full.filter(
        pl.col("first_trip_datetime") >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT"))
    )
    .group_by(
        [
            pl.col("first_trip_datetime").dt.truncate("1w").alias("week"),
            "is_near_station_fmt",
        ]
    )
    .agg(pl.len())
    .sort(pl.col("week")),
    x="week",
    y="len",
    color="is_near_station_fmt",
    color_discrete_map=colors_map,
    category_orders={"is_near_station_fmt": category_order},
    labels={**labels_map, "len": "Nombre de nouveaux conducteurs"},
    template="simple_white",
    title="Evolution de l'acquisition des conducteurs",
    barmode="group",
)
fig_new_drivers_count_by_week.show()

fig_new_drivers_count_by_week.write_html(
    OUTPUT_PATH / "fig_conducteurs_par_semaine.html"
)
fig_new_drivers_count_by_week.write_image(
    OUTPUT_PATH / "fig_conducteurs_par_semaine.svg", width=1280, height=720
)

## Nombre de trajets


In [ ]:
df_journeys_raw_with_nearest_stations_full.filter(
    pl.col("first_trip_datetime") >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT")),
    pl.col("first_trip_datetime") <= datetime.now(ZoneInfo("GMT")) - timedelta(days=14),
).group_by(
    [
        "is_near_station_fmt",
        "driver_identity_key",
        pl.col("start_datetime").dt.truncate("1w"),
    ]
).agg(
    pl.len().alias("num_journeys"),
    pl.concat_str(pl.col("operator_id"), pl.lit("-"), pl.col("operator_trip_id"))
    .n_unique()
    .alias("num_trips"),
).group_by(["is_near_station_fmt", "start_datetime"]).agg(
    pl.col("num_journeys").mean().alias("Nombre moyen de journeys par semaine"),
    pl.col("num_trips").mean().alias("Nombre moyen de trips par semaine"),
).group_by(["is_near_station_fmt"]).agg(
    pl.col("Nombre moyen de journeys par semaine").mean(),
    pl.col("Nombre moyen de trips par semaine").mean(),
)

In [ ]:
df_journeys_raw_with_nearest_stations_full.filter(
    pl.col("first_trip_datetime") >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT")),
    pl.col("first_trip_datetime") <= datetime.now(ZoneInfo("GMT")) - timedelta(days=30),
    pl.col("start_datetime") <= pl.col("first_trip_datetime") + pl.duration(days=30),
).group_by(["is_near_station_fmt", "driver_identity_key"]).agg(
    pl.len().alias("num_journeys"),
    pl.concat_str(pl.col("operator_id"), pl.lit("-"), pl.col("operator_trip_id"))
    .n_unique()
    .alias("num_trips"),
).group_by("is_near_station_fmt").agg(
    pl.col("num_journeys").mean().alias("Nombre moyen de journeys sur 30 jours"),
    pl.col("num_trips").mean().alias("Nombre moyen de trips sur 30 jours"),
)

In [ ]:
def create_num_drivers_by_num_trips_hist_fig(
    df: pl.DataFrame, step_size: int, max_step: int
) -> go.Figure:
    breaks = range(1, max_step + 1, step_size)

    station_distance_categories = (
        df_journeys_raw_with_nearest_stations_full.select("is_near_station_fmt")
        .unique()
        .to_series()
        .to_list()
    )

    # Création du DataFrame de toutes les combinaisons possibles
    combinations = pl.DataFrame(
        product(station_distance_categories, breaks),
        schema=["is_near_station_fmt", "breaks_raw"],
    ).with_columns(
        pl.col("breaks_raw")
        .cut(breaks, include_breaks=True, left_closed=True)
        .struct.unnest()
    )
    data_agg = (
        df.filter(
            pl.col("first_trip_datetime")
            >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT")),
            pl.col("first_trip_datetime")
            <= datetime.now(ZoneInfo("GMT")) - timedelta(days=30),
            pl.col("start_datetime")
            <= pl.col("first_trip_datetime") + pl.duration(days=30),
        )
        .group_by(["is_near_station_fmt", "driver_identity_key"])
        .agg(
            pl.len().alias("num_journeys"),
            pl.concat_str(
                pl.col("operator_id"), pl.lit("-"), pl.col("operator_trip_id")
            )
            .n_unique()
            .alias("num_trips"),
        )
        .with_columns(
            pl.col("num_trips").cut(
                breaks=breaks, left_closed=True, include_breaks=True
            )
        )
        .group_by(["is_near_station_fmt", "num_trips"])
        .agg(pl.col("driver_identity_key").n_unique().alias("num_drivers"))
        .with_columns(pl.col("num_trips").struct.unnest())
    )

    data_complete = (
        combinations.join(
            data_agg,
            on=["is_near_station_fmt", "breakpoint"],
            how="left",
        )
        .with_columns(pl.col("num_drivers").fill_null(0))
        .with_columns(
            (
                100
                * pl.col("num_drivers")
                / pl.col("num_drivers").sum().over("is_near_station_fmt")
            )
            .round(2)
            .alias("share_drivers")
        )
        .sort(["breakpoint"])
    )

    fig = px.bar(
        data_complete,
        x=data_complete["category"],
        y=data_complete["share_drivers"],
        color="is_near_station_fmt",
        color_discrete_map=colors_map,
        category_orders={"is_near_station_fmt": category_order},
        barmode="group",
        template="simple_white",
        title="Distribution du nombre de trips effectués sur 30 jours pour chaque catégorie de distance à une gare",
    )

    fig.update_xaxes(title="Nombre de trajets")
    fig.update_yaxes(title="% des conducteurs")

    return fig, data_complete


fig_drivers_by_trip_numbers_hist, data_complete = (
    create_num_drivers_by_num_trips_hist_fig(
        df_journeys_raw_with_nearest_stations_full, step_size=3, max_step=30
    )
)
fig_drivers_by_trip_numbers_hist.show()

fig_drivers_by_trip_numbers_hist.write_html(
    OUTPUT_PATH / "fig_histo_trajets_conducteurs.html"
)
fig_drivers_by_trip_numbers_hist.write_image(
    OUTPUT_PATH / "fig_histo_trajets_conducteurs.svg", width=1280, height=720
)

## Types de trajets


In [ ]:
df_journeys_raw_with_nearest_stations_full.filter(
    pl.col("first_trip_datetime") >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT")),
    pl.col("first_trip_datetime") <= datetime.now(ZoneInfo("GMT")) - timedelta(days=30),
    pl.col("start_datetime") <= pl.col("first_trip_datetime") + pl.duration(days=30),
).group_by(["is_near_station_fmt", "driver_identity_key"]).agg(
    (
        (
            pl.concat_str(
                pl.col("operator_id"), pl.lit("-"), pl.col("operator_trip_id")
            )
            .filter(pl.col("is_fully_inside_campaign_area"))
            .n_unique()
        )
        >= (
            pl.concat_str(
                pl.col("operator_id"), pl.lit("-"), pl.col("operator_trip_id")
            )
            .filter(pl.col("is_fully_inside_campaign_area").not_())
            .n_unique()
        )
    ).alias("is_intra_driver")
).group_by(["is_near_station_fmt"]).agg(
    (100 * pl.col("is_intra_driver").sum() / pl.len()).alias(
        "% des conducteurs avec une majorité de journeys intra"
    )
)

In [ ]:
df_journeys_trips_count_by_trip_type = (
    df_journeys_raw_with_nearest_stations_full.filter(
        pl.col("first_trip_datetime") >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT")),
        pl.col("first_trip_datetime")
        <= datetime.now(ZoneInfo("GMT")) - timedelta(days=30),
        pl.col("start_datetime")
        <= pl.col("first_trip_datetime") + pl.duration(days=30),
    )
    .group_by(["is_near_station_fmt", "driver_identity_key"])
    .agg(
        (
            (
                pl.concat_str(
                    pl.col("operator_id"), pl.lit("-"), pl.col("operator_trip_id")
                )
                .filter(pl.col("is_fully_inside_campaign_area"))
                .n_unique()
            )
            >= (
                pl.concat_str(
                    pl.col("operator_id"), pl.lit("-"), pl.col("operator_trip_id")
                )
                .filter(pl.col("is_fully_inside_campaign_area").not_())
                .n_unique()
            )
        ).alias("is_intra_driver"),
        pl.len().alias("num_journeys"),
        pl.concat_str(pl.col("operator_id"), pl.lit("-"), pl.col("operator_trip_id"))
        .n_unique()
        .alias("num_trips"),
    )
    .group_by(["is_near_station_fmt", "is_intra_driver"])
    .agg(
        pl.col("num_journeys").mean().alias("Nombre moyen de journeys sur 30 jours"),
        pl.col("num_trips").mean().alias("Nombre moyen de trips sur 30 jours"),
    )
)
df_journeys_trips_count_by_trip_type

In [ ]:
fig_journeys_count_by_driver_type_station_proximity = px.bar(
    df_journeys_trips_count_by_trip_type.with_columns(
        pl.when(pl.col("is_intra_driver"))
        .then(pl.lit("Conducteur intra"))
        .otherwise(pl.lit("Conducteur inter"))
        .alias("driver_type")
    ),
    x="is_near_station_fmt",
    y="Nombre moyen de journeys sur 30 jours",
    color="driver_type",
    text="Nombre moyen de journeys sur 30 jours",
    text_auto=".1f",
    template="simple_white",
    barmode="group",
    labels=labels_map,
    title="Nombre de journeys par type de conducteur et distance d'une gare",
)
fig_journeys_count_by_driver_type_station_proximity.update_layout(legend_title=None)
fig_journeys_count_by_driver_type_station_proximity.show()
fig_journeys_count_by_driver_type_station_proximity.write_html(
    OUTPUT_PATH / "fig_journeys_par_type_conducteur_et_distance_gare.html"
)
fig_journeys_count_by_driver_type_station_proximity.write_image(
    OUTPUT_PATH / "fig_journeys_par_type_conducteur_et_distance_gare.svg",
    width=1280,
    height=720,
)

In [ ]:
fig_trips_count_by_driver_type_station_proximity = px.bar(
    df_journeys_trips_count_by_trip_type.with_columns(
        pl.when(pl.col("is_intra_driver"))
        .then(pl.lit("Conducteur intra"))
        .otherwise(pl.lit("Conducteur inter"))
        .alias("driver_type")
    ),
    x="is_near_station_fmt",
    y="Nombre moyen de trips sur 30 jours",
    color="driver_type",
    text="Nombre moyen de trips sur 30 jours",
    text_auto=".1f",
    template="simple_white",
    barmode="group",
    labels=labels_map,
    title="Nombre de trips par type de conducteur et distance d'une gare",
)

fig_trips_count_by_driver_type_station_proximity.update_layout(legend_title=None)
fig_trips_count_by_driver_type_station_proximity.show()
fig_trips_count_by_driver_type_station_proximity.write_html(
    OUTPUT_PATH / "fig_trips_par_type_conducteur_et_distance_gare.html"
)
fig_trips_count_by_driver_type_station_proximity.write_image(
    OUTPUT_PATH / "fig_trips_par_type_conducteur_et_distance_gare.svg",
    width=1280,
    height=720,
)

In [ ]:
df_passenger_mean_by_distance_cat = (
    df_journeys_raw_with_nearest_stations_full.filter(
        pl.col("first_trip_datetime") >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT")),
        pl.col("first_trip_datetime")
        <= datetime.now(ZoneInfo("GMT")) - timedelta(days=30),
        pl.col("start_datetime")
        <= pl.col("first_trip_datetime") + pl.duration(days=30),
    )
    .group_by(
        [
            pl.concat_str(
                pl.col("operator_id"), pl.lit("-"), pl.col("operator_trip_id")
            ).alias("trip_id"),
        ]
    )
    .agg(
        pl.col("is_fully_inside_campaign_area").max(),
        pl.col("passenger_seats").sum(),
        pl.col("is_near_station_fmt").mode().first().alias("is_near_station_fmt"),
        pl.col("driver_identity_key").max(),
    )
    .group_by(["is_near_station_fmt", "driver_identity_key"])
    .agg(
        (
            (
                pl.col("trip_id")
                .filter(pl.col("is_fully_inside_campaign_area"))
                .n_unique()
            )
            >= (
                pl.col("trip_id")
                .filter(pl.col("is_fully_inside_campaign_area").not_())
                .n_unique()
            )
        ).alias("is_intra_driver"),
        pl.len().alias("num_journeys"),
        pl.col("passenger_seats").mean(),
    )
    .group_by(["is_near_station_fmt", "is_intra_driver"])
    .agg(
        pl.col("passenger_seats").mean().alias("Nombre moyen de passagers"),
    )
)
df_passenger_mean_by_distance_cat

In [ ]:
fig_passengers_count_by_driver_type_distance_gare = px.bar(
    df_passenger_mean_by_distance_cat.with_columns(
        pl.when(pl.col("is_intra_driver"))
        .then(pl.lit("Conducteur intra"))
        .otherwise(pl.lit("Conducteur inter"))
        .alias("driver_type")
    ),
    x="is_near_station_fmt",
    y="Nombre moyen de passagers",
    color="driver_type",
    text="Nombre moyen de passagers",
    text_auto=".2f",
    template="simple_white",
    barmode="group",
    labels=labels_map,
    title="Nombre moyen de passagers par type de conducteur et campagne",
)

fig_passengers_count_by_driver_type_distance_gare.update_layout(legend_title=None)
fig_passengers_count_by_driver_type_distance_gare.show()
fig_passengers_count_by_driver_type_distance_gare.write_html(
    OUTPUT_PATH / "fig_passagers_type_conducteur_et_distance_gare.html"
)
fig_passengers_count_by_driver_type_distance_gare.write_image(
    OUTPUT_PATH / "fig_passagers_par_type_conducteur_et_distance_gare.svg",
    width=1280,
    height=720,
)

## Rétention


In [ ]:
df_acquisition_by_driver_type = (
    (
        df_journeys_raw_with_nearest_stations_full.filter(
            pl.col("first_trip_datetime")
            >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT")),
            pl.col("first_trip_datetime")
            <= datetime.now(ZoneInfo("GMT")) - timedelta(weeks=6),
        )
        .group_by(["driver_identity_key"])
        .agg(
            pl.col("start_datetime").min(),
            pl.datetime_range(
                pl.col("start_datetime").min().dt.truncate("1w"),
                pl.col("start_datetime").min().dt.truncate("1w") + pl.duration(weeks=5),
                "1w",
            ).alias("week"),
            pl.col("is_near_station_fmt").mode().first().alias("is_near_station_fmt"),
        )
        .explode("week")
        .join(
            df_journeys_raw_with_nearest_stations_full.filter(
                pl.col("start_datetime")
                >= datetime(2024, 9, 1, tzinfo=ZoneInfo("GMT")),
            ),
            left_on=["driver_identity_key", "week"],
            right_on=[
                "driver_identity_key",
                pl.col("start_datetime").dt.truncate("1w"),
            ],
            how="left",
        )
    )
    .group_by([pl.col("driver_identity_key"), "week"])
    .agg(
        (pl.col("_id").count() > 0).alias("has_traveled"),
        pl.col("is_near_station_fmt").mode().first().alias("is_near_station_fmt"),
    )
    .with_columns(
        pl.col("week")
        .rank()
        .over(partition_by=["driver_identity_key"], order_by="week")
        .alias("week_number")
    )
    .group_by(["is_near_station_fmt", "week_number"])
    .agg(
        (100 * pl.col("has_traveled").sum() / pl.col("has_traveled").count()).alias(
            "drivers_share"
        )
    )
    .sort([pl.col("week_number")])
)
df_acquisition_by_driver_type

In [ ]:
fig_churn_by_campaign_type = px.line(
    df_acquisition_by_driver_type,
    x="week_number",
    y="drivers_share",
    color="is_near_station_fmt",
    color_discrete_map=colors_map,
    category_orders={"is_near_station_fmt": category_order},
    template="simple_white",
    labels=labels_map,
    title="Attrition en fonction des différentes types de campagne",
)

fig_churn_by_campaign_type.update_yaxes(showgrid=True)
fig_churn_by_campaign_type.show()

fig_churn_by_campaign_type.write_html(OUTPUT_PATH / "fig_attrition_par_campagne.html")
fig_churn_by_campaign_type.write_image(
    OUTPUT_PATH / "fig_attrition_par_campagne.svg", width=1280, height=720
)

## Distribution des gains


In [ ]:
df_incentives_stats_by_month_driver = (
    df_journeys_raw_with_nearest_stations_full.with_columns(
        (pl.col("incentive_amount") / 100)
        .cum_sum()
        .over(
            partition_by=[
                "driver_identity_key",
                pl.col("start_datetime").dt.truncate("1mo"),
            ],
            order_by="start_datetime",
        )
        .alias("incentive_amount_cumu")
    )
    .group_by([pl.col("start_datetime").dt.truncate("1mo"), "driver_identity_key"])
    .agg(
        pl.col("incentive_amount_cumu").max().alias("incentive_amount_cumu_max"),
        pl.col("is_near_station_fmt").mode().first().alias("is_near_station_fmt"),
    )
)

In [ ]:
breaks = list(range(0, 101, 10))
df_incentives_by_drivers_hist = (
    df_incentives_stats_by_month_driver.group_by(
        [
            pl.col("incentive_amount_cumu_max").cut(breaks=breaks, include_breaks=True),
            "is_near_station_fmt",
        ]
    )
    .agg(pl.col("driver_identity_key").n_unique().alias("driver_count"))
    .with_columns(
        pl.col("incentive_amount_cumu_max").struct.unnest(),
        (
            100
            * pl.col("driver_count")
            / pl.col("driver_count").sum().over("is_near_station_fmt")
        )
        .round(2)
        .alias("share"),
    )
    .sort(["is_near_station_fmt", "breakpoint"])
)

fig_incentives_by_driver_hist = px.bar(
    df_incentives_by_drivers_hist,
    x="category",
    y="share",
    text="share",
    color="is_near_station_fmt",
    color_discrete_map=colors_map,
    category_orders={"is_near_station_fmt": category_order},
    barmode="group",
    labels={**labels_map, "share": "% des conducteurs", "category": "Incitation reçue"},
    template="simple_white",
    title="Distribution des gains mensuels des conducteurs<br><sub>Par tranche de 10€, la première tranche est celle des conducteurs n'ayant perçu aucune incitation.</sub>",
)
fig_incentives_by_driver_hist.show()
fig_incentives_by_driver_hist.write_html(
    OUTPUT_PATH / "fig_histo_incitation_conducteur.html"
)
fig_incentives_by_driver_hist.write_image(
    OUTPUT_PATH / "fig_histo_incitation_conducteur.svg", width=1280, height=720
)